<a href="https://colab.research.google.com/github/Shilaku-f1/swahili-nlp-model/blob/main/swahili_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Required Packages**

In [ ]:
#1 INSTALL REQUIRED LIBRARIES

!pip install keras==2.15.0
!pip install tensorboard==2.15.0
!pip install ml-dtypes==0.2.0
!pip install transformers datasets torch sentencepiece
!pip install tensorflow
!pip install accelerate>=0.26.0
!pip list

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.2 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires keras>=3.2.0, but you have keras 2.15.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 28.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
ERROR: pip's de

Set Environment Variables to Avoid Errors

In [ ]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"
# os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"  # Uncomment this to extend memory allocation (may hang your computer)


Verify Current Directory

In [ ]:
!pwd


/content


# **Load the Dataset**

In [ ]:
from datasets import load_dataset

# Replace 'swahili2.txt' with the correct path to your dataset
dataset = load_dataset('text', data_files='swahili2.txt')

# Display the first 5 rows of the dataset
print(dataset['train'].select(range(5)))

# Alternatively, print the text field directly
for i in range(5):
    print(dataset['train'][i]['text'])


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 5
})

Bulaya amesota rumande tangu akamatwe Ijumaa ya Agosti 18 kwa madai ya kuingia Jimbo la Tarime na kufanya shughuli za kisiasa kwa kutoa msaada wa mifuko 50 ya saruji kwa ajili ya ujenzi wa miundombinu katika shule za msingi jimboni humo baada ya kualikwa na mbunge mwenzake Ester Matiko.
Kwani kwa upande wa Polisi walidai kwamba wakati wnmuuwa Matheri alitoka akiwa na bunduki aina ya AK-47 na hivyo kuwalazimisha Polisi kumuuwa ili kujihami, lakini kwa upande wa mkewe, Felister Wanjiru ambaye alikuwa na ujauzito wa miezi saba wakati huo, akihojiwa na kituo cha Luninga cha KTN alidai kuwa mumewe alitoka akiwa hana silaha yoyote, tena alitoka akiwa kifua wazi akiwa amevaa kaptula, maarufu kama Bukta huku akiwa ameweka mikono yake kichwani ili kuwaonesha Polisi kwamba hakuwa na silaha yoyote, lakini kwa mshangao Polisi walimmiminia risasi mumewe na kumuuwa palepale.
Serikali ya nchi hiyo imetangaza tahafifu, ikiwemo ongezeko la asilimia 

# **Use a Pre-existing Tokenizer**

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
print("DONE")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

DONE


**Tokenize the Dataset**

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
print("DONE")


Map:   0%|          | 0/21245 [00:00<?, ? examples/s]

DONE


# **Create Data Collector for Language Modeling**

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True)
print("DONE")


DONE


**Configure Training Arguments**

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./swahili_model",
    eval_strategy="no",  # No evaluation during training
    learning_rate=1e-5,
    weight_decay=0.01,
    num_train_epochs=1,
    per_device_train_batch_size=8,
)
print("DONE")


DONE


**Load the Pre-trained Model**

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased", trust_remote_code=True)
print("Model loaded successfully.")


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

Model loaded successfully.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Train the Model**

**Checking availability of GPU**

In [ ]:
import torch
print("GPU available:", torch.cuda.is_available())


GPU available: True


In [ ]:
from transformers import Trainer, TrainingArguments
from datasets import DatasetDict

# Manually split the dataset into train and test
split = tokenized_dataset["train"].train_test_split(test_size=0.1)
tokenized_dataset = DatasetDict({
    "train": split["train"],
    "test": split["test"]
})

# Define TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Or keep "evaluation_strategy" for older versions
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Start Training
trainer.train()
print("DONE")


<ipython-input-12-bd63117d6670>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,2.663100,2.325922
2,2.218400,2.077002
3,2.023700,1.973319


DONE


# **Saving the model**

In [ ]:
model.save_pretrained("./swahili_model")
tokenizer.save_pretrained("./swahili_model")


('./swahili_model/tokenizer_config.json',
 './swahili_model/special_tokens_map.json',
 './swahili_model/vocab.txt',
 './swahili_model/added_tokens.json',
 './swahili_model/tokenizer.json')

# **Evaluate the model**

In [ ]:
from transformers import pipeline

# Load the fine-tuned model
fill_mask = pipeline("fill-mask", model="./swahili_model", tokenizer="./swahili_model")

# Test the model with a sample Swahili sentence
result = fill_mask("Wakati [MASK] leo.")  # Replace with any Swahili text
print(result)


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'score': 0.8899622559547424, 'token': 11333, 'token_str': 'wa', 'sequence': 'wakati wa leo.'}, {'score': 0.06053682416677475, 'token': 6583, 'token_str': 'na', 'sequence': 'wakati na leo.'}, {'score': 0.012930585071444511, 'token': 9152, 'token_str': 'ni', 'sequence': 'wakati ni leo.'}, {'score': 0.006846090778708458, 'token': 1010, 'token_str': ',', 'sequence': 'wakati, leo.'}, {'score': 0.004417065065354109, 'token': 1011, 'token_str': '-', 'sequence': 'wakati - leo.'}]
